Experimenting in order to ultimately compute the cell structures of Voronoi complexes

In [9]:
using Pkg
Pkg.activate(normpath(joinpath(@__DIR__, "../")))
Pkg.add("Polyhedra")
Pkg.add("QHull")
using Revise
using LinearAlgebra
using Polyhedra
using QHull
import CDDLib
ENV["JULIA_NUM_THREADS"] = Sys.CPU_THREADS÷2
LinearAlgebra.BLAS.set_num_threads(Sys.CPU_THREADS÷2)
using Serialization

using SLnCohomology

  Activating project at `/mnt/c/Git/HigherTSL3/SLnCohomology`


   Resolving package versions...


  No Changes to `/mnt/c/Git/HigherTSL3/SLnCohomology/Project.toml`
  No Changes to `/mnt/c/Git/HigherTSL3/SLnCohomology/Manifest.toml`


   Resolving package versions...


  No Changes to `/mnt/c/Git/HigherTSL3/SLnCohomology/Project.toml`
  No Changes to `/mnt/c/Git/HigherTSL3/SLnCohomology/Manifest.toml`


# Test space for Voronoi cells

In [3]:
#SL_3

e1 = [1,0,0]
e2 = [0,1,0]
e3 = [0,0,1]
e12 = [1,1,0]
e13 = [1,0,1]
e123 = [1,1,1]
m2 = [e1 e2 e3] # standard
m3_1 = [e1 e2 e3 e12] # 2-additive
m3_2 = [e1 e2 e3 e123] # 3-additive
m4 = [e1 e2 e3 e12 e13] # double-triple
m5 = [e1 e2 e3 e12 e13 e123] # Q
cells_SL3 = Dict()
cells_SL3[2] = [m2]
cells_SL3[3] = [m3_1,m3_2]
cells_SL3[4] = [m4]
cells_SL3[5] = [m5]

1-element Vector{Matrix{Int64}}:
 [1 0 … 1 1; 0 1 … 0 1; 0 0 … 1 1]

In [8]:
dim_4_cells = []
min_vectors_facet(m5,dim_4_cells)

1-element Vector{Any}:
 [0 0 … 1 1; 1 0 … 0 1; 0 1 … 1 1]

In [9]:
dim_3_cells = []
min_vectors_facet(dim_4_cells[1],dim_3_cells)

2-element Vector{Any}:
 [0 1 1 1; 0 1 0 1; 1 0 1 1]
 [0 0 1 1; 1 0 1 0; 0 1 0 1]

In [10]:
dim_2_cells = []
min_vectors_facet(dim_3_cells[1],dim_2_cells)
min_vectors_facet(dim_3_cells[2],dim_2_cells)

2-element Vector{Any}:
 [1 1 1; 1 0 1; 0 1 1]
 [0 1 1; 0 1 1; 1 0 1]

In [11]:
dim_2_cells[1]

3×3 transpose(::Matrix{Int64}) with eltype Int64:
 1  1  1
 1  0  1
 0  1  1

In [16]:
SLnCohomology.quadratic_form(dim_2_cells[1])

3×3 Matrix{Int64}:
 3  2  2
 2  2  1
 2  1  2

In [12]:
dim_2_cells[2]

3×3 transpose(::Matrix{Int64}) with eltype Int64:
 0  1  1
 0  1  1
 1  0  1

In [13]:
m = SLnCohomology.quadratic_form(dim_2_cells[2])

3×3 Matrix{Int64}:
 2  2  1
 2  2  1
 1  1  2

In [14]:
isposdef(m)

true

In [15]:
dim_2_cells[2]*transpose(dim_2_cells[2])

3×3 Matrix{Int64}:
 2  2  1
 2  2  1
 1  1  2

## Checking
maybe Dan gave me the data in a slightly different format. I should retry with the original data.

# Cleaned up version

In [41]:
# From sl_cell_computations, no need to have it twice in the end
function same_orbit(matrix1,matrix2)
    #= Checks whether matrix1 lies in the SL_n orbit as matrix2
    =#
    if length(SLnCohomology.stabiliser_coset_SL(SLnCohomology.quadratic_form(matrix1),SLnCohomology.quadratic_form(matrix2))) > 0
        return true
    else
        return false
    end
end

function orbit_in_list(matrix,list)
    #= Checks whether matrix lies in the SL_n orbit of one of the elements in list
    =#
    for matrix2 in list
        if same_orbit(matrix,matrix2)
            return true
        end
    end
    return false
end

orbit_in_list (generic function with 1 method)

In [42]:
function create_polyhedron(cell)
    poly_vertices = Vector{Int64}[]
    for col in eachcol(cell)
        push!(poly_vertices,vec(SLnCohomology.quadratic_form(col)))
    end
    poly_cell = polyhedron(vrep(poly_vertices),DefaultLibrary{Float64}(GLPK.Optimizer))
    return poly_cell
end

create_polyhedron (generic function with 1 method)

In [43]:
function facets(polyhedral_cell, min_vectors, codim_1_cells)
    #=
    min_vectors must be ordered in the same way as the points in polyhedral_cell
    =#
    vertex_list = collect(points(polyhedral_cell))
    for halfspace in eachindex(halfspaces(polyhedral_cell))
        # create a list with all min_vectors that lie on this facet
        vertices_facet = []
        for vertex in incidentpoints(polyhedral_cell, halfspace) # read out using Oscar/polymake in the long term, this might get shorter
            # also careful: At the moment, I'm not reducing the cells, this should be done or avoided by using polymake
            vertex_index = findfirst(x -> x==vertex, vertex_list) # this should somehow get quicker, but I couldn't find out how
            push!(vertices_facet,min_vectors[vertex_index])
        end
        #now turn into matrix again to make accessible to other calculations, orbits extract_basis
        facet = transpose(reduce(vcat,transpose.(vertices_facet)))
        if isposdef(SLnCohomology.quadratic_form(facet))
            # then it intersects the interior non-trivially - make this a separate function to increase readability
            if !orbit_in_list(facet, codim_1_cells)
                push!(codim_1_cells,facet)
            end
        end
    end
    return codim_1_cells
end

facets (generic function with 1 method)

In [44]:
function min_vectors_facet(cell,codim_1_cells)
    min_vectors = collect(eachcol(cell))
    return facets(create_polyhedron(cell), min_vectors, codim_1_cells)
end

min_vectors_facet (generic function with 1 method)

In [45]:
function Voronoi_cells(n,perfect_forms)
    #= perfect_forms: list of perfect forms in dimension n, up to action of SL/ GL
    =#
    dim_symmetric_space = n*(n+1)/2-1
    cells_SLn = Dict()
    perfect_forms_min_vec_rep = []
    for perfect_form in perfect_forms
        # compute the minimal vectors and put them in the right format
        push!(perfect_forms_min_vec_rep, transpose(reduce(vcat,transpose.(SLnCohomology.minimal_vectors(perfect_form)))))
        println("I computed minimal vectors.")
    end
    cells_SLn[dim_symmetric_space] = perfect_forms_min_vec_rep # the perfect forms are the top cells
    for dim in dim_symmetric_space-1:-1:n-1 # these are the dimensions where there are cells that intersect the interior non-trivially
        println("I'm computing in dimension $dim")
        cells_SLn[dim] = []
        for cell in cells_SLn[dim+1]
            min_vectors_facet(cell,cells_SLn[dim]) # function needs better name
        end
    end
    return cells_SLn
end

Voronoi_cells (generic function with 1 method)

In [46]:
A_3 = [2 -1 0
-1 2 -1
0 -1 2]
forms_3 = [A_3]

1-element Vector{Matrix{Int64}}:
 [2 -1 0; -1 2 -1; 0 -1 2]

In [47]:
cells_SL3 = Voronoi_cells(3,forms_3)

I computed minimal vectors.
I'm computing in dimension 4.0


I'm computing in dimension 3.0
I'm computing in dimension 2.0


Dict{Any, Any} with 4 entries:
  5.0 => Any[[1 0 … 0 1; 1 1 … 1 0; 1 1 … 0 0]]
  4.0 => Any[[0 0 … 0 1; 1 0 … 1 0; 1 1 … 0 0]]
  2.0 => Any[[0 0 1; 0 1 0; 1 0 0]]
  3.0 => Any[[0 1 0 1; 0 1 1 0; 1 0 0 0], [0 0 1 1; 1 0 1 0; 1 1 0 0]]

In [48]:
D_4 = [2 0 1 0
0 2 -1 0
1 -1 2 -1
0 0 -1 2]

A_4 = [2 -1 0 0
-1 2 -1 0
0 -1 2 -1
0 0 -1 2]

forms_4 = [D_4, A_4]

2-element Vector{Matrix{Int64}}:
 [2 0 1 0; 0 2 -1 0; 1 -1 2 -1; 0 0 -1 2]
 [2 -1 0 0; -1 2 -1 0; 0 -1 2 -1; 0 0 -1 2]

In [49]:
cells_SL4 = Voronoi_cells(4,forms_4)

I computed minimal vectors.
I computed minimal vectors.
I'm computing in dimension 8.0


I'm computing in dimension 7.0
I'm computing in dimension 6.0
I'm computing in dimension 5.0
I'm computing in dimension 4.0


I'm computing in dimension 3.0


Dict{Any, Any} with 7 entries:
  5.0 => Any[[0 -1 … 0 1; 0 1 … 1 0; 0 1 … 0 0; 1 0 … 0 0], [-1 0 … 0 1; 0 0 … …
  4.0 => Any[[0 0 … 0 1; 0 0 … 1 0; 0 1 … 0 0; 1 0 … 0 0], [0 -1 … 0 1; 0 1 … 1…
  6.0 => Any[[0 0 … 0 1; 0 1 … 1 0; 0 1 … 0 0; 1 0 … 0 0], [-1 0 … 0 1; 0 0 … 1…
  7.0 => Any[[-1 0 … 0 1; 0 0 … 1 0; 1 0 … 0 0; 1 1 … 0 0], [0 -1 … 0 1; 0 0 … …
  9.0 => Any[[-1 0 … 0 1; 1 1 … 1 0; 2 1 … 0 0; 1 1 … 0 0], [1 0 … 0 1; 1 1 … 1…
  8.0 => Any[[0 -1 … 0 1; 0 0 … 1 0; 1 1 … 0 0; 1 1 … 0 0], [0 0 … 0 1; 1 0 … 1…
  3.0 => Any[[0 -1 0 1; 0 0 1 0; 0 1 0 0; 1 0 0 0]]

In [50]:
D_5 = [2 0 1 0 0
0 2 -1 0 0
1 -1 2 -1 0
0 0 -1 2 -1
0 0 0 -1 2]

A_5_plus3 = [6 -3 0 0 0
-3 6 -3 0 3
0 -3 6 -3 0
0 0 -3 6 0
0 3 0 0 4]

A_5 = [2 -1 0 0 0
-1 2 -1 0 0
0 -1 2 -1 0
0 0 -1 2 -1
0 0 0 -1 2]

forms_5 = [D_5, A_5_plus3, A_5]

3-element Vector{Matrix{Int64}}:
 [2 0 … 0 0; 0 2 … 0 0; … ; 0 0 … 2 -1; 0 0 … -1 2]
 [6 -3 … 0 0; -3 6 … 0 3; … ; 0 0 … 6 0; 0 3 … 0 4]
 [2 -1 … 0 0; -1 2 … 0 0; … ; 0 0 … 2 -1; 0 0 … -1 2]

In [51]:
lib = QHull.Library()

QHull.Library(nothing)

In [33]:
Pkg.add("CDDLib")
Pkg.add("GLPK")

   Resolving package versions...


  No Changes to `/mnt/c/Git/HigherTSL3/SLnCohomology/Project.toml`
  No Changes to `/mnt/c/Git/HigherTSL3/SLnCohomology/Manifest.toml`


   Resolving package versions...


   Installed GLPK ─ v1.1.3


    Updating `/mnt/c/Git/HigherTSL3/SLnCohomology/Project.toml`
  [60bf3e95] + GLPK v1.1.3
    Updating `/mnt/c/Git/HigherTSL3/SLnCohomology/Manifest.toml`
 

 [60bf3e95] + GLPK v1.1.3
  [e8aa6df9] + GLPK_jll v5.0.1+0


Precompiling project...


  ✓ GLPK


  ✓ SLnCohomology


  2 dependencies successfully precompiled in 47 seconds. 165 already precompiled.
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version


In [40]:
import CDDLib
import GLPK

In [18]:
lib = CDDLib.Library()

CDDLib.Library(:float)

In [52]:
cells_SL5 = Voronoi_cells(5,forms_5)

I computed minimal vectors.
I computed minimal vectors.
I computed minimal vectors.
I'm computing in dimension 13.0


I'm computing in dimension 12.0


I'm computing in dimension 11.0


I'm computing in dimension 10.0


I'm computing in dimension 9.0


...

I'm computing in dimension 7.0


I'm computing in dimension 6.0


I'm computing in dimension 5.0


I'm computing in dimension 4.0


Dict{Any, Any} with 11 entries:
  5.0  => Any[[0 0 … 0 1; 0 0 … 1 0; … ; 0 1 … 0 0; 1 0 … 0 0], [0 0 … 0 1; 0 0…
  12.0 => Any[[-1 0 … 0 1; 0 0 … 1 0; … ; 1 1 … 0 0; 1 1 … 0 0], [0 -1 … 0 1; 0…
  8.0  => Any[[0 -1 … 0 1; 0 0 … 1 0; … ; 0 1 … 0 0; 1 0 … 0 0], [0 0 … 0 1; 0 …
  6.0  => Any[[0 0 … 0 1; 0 0 … 1 0; … ; 0 1 … 0 0; 1 0 … 0 0], [0 -1 … 0 1; 0 …
  11.0 => Any[[0 0 … 0 1; 0 0 … 1 0; … ; 1 0 … 0 0; 1 1 … 0 0], [-1 0 … 0 1; 0 …
  9.0  => Any[[0 0 … 0 1; 0 0 … 1 0; … ; 0 1 … 0 0; 1 0 … 0 0], [0 0 … 0 1; 0 1…
  14.0 => Any[[-1 -1 … 0 1; 1 1 … 1 0; … ; 2 1 … 0 0; 1 1 … 0 0], [2 1 … 1 0; 3…
  7.0  => Any[[0 0 … 0 1; 0 0 … 1 0; … ; 0 1 … 0 0; 1 0 … 0 0], [0 -1 … 0 1; 0 …
  13.0 => Any[[0 -1 … 0 1; 0 0 … 1 0; … ; 1 1 … 0 0; 1 1 … 0 0], [-1 0 … 0 1; 1…
  4.0  => Any[[0 0 … 0 1; 0 0 … 1 0; … ; 0 1 … 0 0; 1 0 … 0 0], [0 0 … 0 1; 0 0…
  10.0 => Any[[0 -1 … 0 1; 0 1 … 1 0; … ; 0 1 … 0 0; 1 0 … 0 0], [0 0 … 0 1; 0 …

compare with previous results

In [13]:
cells_SL4_old = deserialize(joinpath(@__DIR__, "./differentials_computation/precomputed_cells/sl4_cells.sjl"))
cells_SL3_old = deserialize(joinpath(@__DIR__, "./differentials_computation/precomputed_cells/sl3_cells.sjl"))
cells_SL5_old = deserialize(joinpath(@__DIR__, "./differentials_computation/precomputed_cells/sl5_cells.sjl"))

Dict{Any, Any} with 11 entries:
  5  => Any[[0 0 … 0 1; 0 0 … 1 -1; … ; 1 0 … 0 0; 0 0 … 0 0], [0 0 … 0 1; 0 0 …
  7  => Any[[0 0 … 1 1; 0 0 … -1 0; … ; 1 0 … 0 0; 0 0 … 0 0], [0 0 … 1 1; 0 0 …
  12 => Any[[0 0 … 1 1; 0 0 … 0 1; … ; 1 1 … 1 0; -1 0 … -1 -1], [0 0 … 1 1; 0 …
  8  => Any[[0 0 … 1 1; 0 0 … 0 0; … ; 1 0 … 0 -1; 0 0 … 0 0], [0 0 … 1 1; 0 0 …
  4  => Any[[0 0 … 0 1; 0 0 … 1 -1; … ; 1 0 … 0 0; 0 0 … -1 0], [0 0 … 1 1; 0 1…
  6  => Any[[0 0 … 0 1; 0 0 … 1 -1; … ; 1 0 … 1 0; 0 0 … -1 0], [0 0 … 1 1; 0 0…
  13 => Any[[0 0 … 1 1; 0 0 … 0 1; … ; 1 1 … 1 0; -1 0 … -1 -1], [0 0 … 1 1; 0 …
  11 => Any[[0 0 … 1 1; 0 0 … 0 1; … ; 1 0 … 0 0; 0 0 … 0 -1], [0 0 … 1 1; 0 0 …
  10 => Any[[0 0 … 1 1; 0 0 … 0 0; … ; 1 0 … 0 0; 0 0 … -1 0], [0 0 … 1 1; 0 0 …
  9  => Any[[0 0 … 1 1; 0 0 … 0 0; … ; 1 0 … -1 0; 0 0 … 0 -1], [0 0 … 1 1; 0 0…
  14 => Any[[0 0 … 1 1; 0 0 … 0 1; … ; 1 1 … 0 0; -1 0 … -1 -1], [0 0 … 1 1; 0 …

In [17]:
for dim in keys(cells_SL3_old)
    for old_cell in cells_SL3_old[dim]
        if orbit_in_list(old_cell,cells_SL3[dim])
            println("Found the cell")
        else
            println("Not there")
        end
    end
end

Found the cell
Found the cell
Found the cell
Found the cell
Found the cell


In [18]:
for dim in keys(cells_SL4_old)
    for old_cell in cells_SL4_old[dim]
        if orbit_in_list(old_cell,cells_SL4[dim])
            println("Found the cell")
        else
            println("Not there")
        end
    end
end

Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell


In [19]:
for dim in keys(cells_SL5_old)
    for old_cell in cells_SL5_old[dim]
        if orbit_in_list(old_cell,cells_SL5[dim])
            println("Found the cell")
        else
            println("Not there")
        end
    end
end

Found the cell
Found the cell
Found the cell


Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell


Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell


Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell


Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell


...

Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell


Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell
Found the cell


Found the cell


Found the cell


Found the cell
Found the cell


In [53]:
E6 = [2 -1 0 0 0 0
-1 2 -1 0 0 0
0 -1 2 -1 0 -1
0 0 -1 2 -1 0
0 0 0 -1 2 0
0 0 -1 0 0 2]

E6star = [4 5 6 4 2 3
5 10 12 8 4 6
6 12 18 12 6 9
4 8 12 10 5 6
2 4 6 5 4 3
3 6 9 6 3 6]

D6 = [2 0 1 0 0 0
0 2 -1 0 0 0
1 -1 2 -1 0 0
0 0 -1 2 -1 0
0 0 0 -1 2 -1
0 0 0 0 -1 2]

A6_2 = [4 1 2 2 2 2
1 4 2 2 2 2
2 2 4 1 1 2
2 2 1 4 1 2
2 2 1 1 4 2
2 2 2 2 2 4]

A6_sup2 = [4 -1 -2 1 1 -2
-1 4 -1 -2 1 1
-2 -1 4 -1 -2 1
1 -2 -1 4 -1 -2
1 1 -2 -1 4 -1
-2 1 1 -2 -1 4]

A6_1 = [4 1 2 2 2 2
1 4 2 2 2 2
2 2 4 1 2 2
2 2 1 4 2 2
2 2 2 2 4 1
2 2 2 2 1 4]

A6 = [2 -1 0 0 0 0
-1 2 -1 0 0 0
0 -1 2 -1 0 0
0 0 -1 2 -1 0
0 0 0 -1 2 -1
0 0 0 0 -1 2]

forms_6 = [E6, E6star,D6, A6_2, A6_sup2, A6_1, A6]

7-element Vector{Matrix{Int64}}:
 [2 -1 … 0 0; -1 2 … 0 0; … ; 0 0 … 2 0; 0 0 … 0 2]
 [4 5 … 2 3; 5 10 … 4 6; … ; 2 4 … 4 3; 3 6 … 3 6]
 [2 0 … 0 0; 0 2 … 0 0; … ; 0 0 … 2 -1; 0 0 … -1 2]
 [4 1 … 2 2; 1 4 … 2 2; … ; 2 2 … 4 2; 2 2 … 2 4]
 [4 -1 … 1 -2; -1 4 … 1 1; … ; 1 1 … 4 -1; -2 1 … -1 4]
 [4 1 … 2 2; 1 4 … 2 2; … ; 2 2 … 4 1; 2 2 … 1 4]
 [2 -1 … 0 0; -1 2 … 0 0; … ; 0 0 … 2 -1; 0 0 … -1 2]

In [54]:
cells_SL6 = Voronoi_cells(6,forms_6)

I computed minimal vectors.
I computed minimal vectors.
I computed minimal vectors.
I computed minimal vectors.
I computed minimal vectors.
I computed minimal vectors.
I computed minimal vectors.
I'm computing in dimension 19.0


I'm computing in dimension 18.0


I'm computing in dimension 17.0


I'm computing in dimension 16.0


I'm computing in dimension 15.0


...

I'm computing in dimension 8.0


I'm computing in dimension 7.0


I'm computing in dimension 6.0


I'm computing in dimension 5.0


ErrorException: Error thrown by GAP: Error, List Element: <list>[6] must have an assigned value in
  m + 1 / 1000 - dam at /home/bbrueck/.julia/artifacts/b5c2f0f824457e5c391fb24916f94d5d91c62c4f/share/gap/lib/zlattice.gi:1274 called from 
srt( n, 0 ); at /home/bbrueck/.julia/artifacts/b5c2f0f824457e5c391fb24916f94d5d91c62c4f/share/gap/lib/zlattice.gi:1355 called from
<function "ShortestVectors">( <arguments> )
 called from read-eval loop at *defin*:0
